In [ ]:
# MSD demo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from trajkit import Trajectory, TrajectorySet, save_trajectory_set, load_trajectory_set
from trajkit.stats import msd, msd_trajectory_set


In [ ]:
# Build synthetic trajectories (2D random walks)
np.random.seed(0)
N = 5  # number of tracks
T = 200  # length per track

trajset = TrajectorySet(dataset_id="msd_demo")
for i in range(N):
    steps = np.random.randn(T, 2)
    x = np.cumsum(steps, axis=0)
    frame = np.arange(T, dtype=int)
    tr = Trajectory(track_id="p{}".format(i), x=x, frame=frame, frame_rate_hz=20.0)
    trajset.add(tr)

trajset.summary_table().head()


In [ ]:
from trajkit import load_trajectory_set
trajset = load_trajectory_set("/Users/mehdi/air-water/data/parquet/experiment_001_2017-08-16/exp001_t043m_r01um_2017-08-16")

In [ ]:
# Save and reload (optional example of Parquet I/O)
import os
out_dir = "results/msd_demo_dataset"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
save_trajectory_set(trajset, out_dir)
trajset = load_trajectory_set(out_dir, frame_rate_hz=20.0)


In [ ]:
trajset

In [ ]:
# MSD for a single trajectory
tr0 = trajset.get("1.0")
msd_single = msd(tr0, max_lag=40, per_dim=True, return_sd=True)
msd_single.head()


In [ ]:
# MSD for the whole set (pair-weighted aggregation)
msd_pair = msd_trajectory_set(
    trajset,
    max_lag=40,
    aggregate=True,
    aggregate_mode="pair",
    per_dim=True,
)
msd_pair.head()


In [ ]:
# Plot MSD
fig, ax = plt.subplots()
ax.plot(msd_pair["lag"], msd_pair["msd"], marker="o", label="pair-weighted msd")
ax.set_xlabel("lag (frames)")
ax.set_ylabel("MSD")
ax.grid(True, linestyle=":", alpha=0.3)
ax.legend()
plt.show()
